# Public Key Encryption
Imagine Alice wants something valuable shipped to her. Because it’s valuable, she wants to make sure it arrives securely (i.e. hasn’t been opened or tampered with) and that it’s not a forgery (i.e. it’s actually from the sender she’s expecting it to be from and nobody’s pulling the old switcheroo).

One way she can do this is by providing the sender (let’s call him Bob) with a high-security box of her choosing. She provides Bob with this box, and something else: a padlock, but a padlock without a key. Alice is keeping that key all to herself. Bob can put items in the box then put the padlock onto it. But once the padlock snaps shut, the box cannot be opened by anyone who doesn’t have Alice’s private key.

Here’s the twist though: Bob also puts a padlock onto the box. This padlock uses a key Bob has published to the world, such that if you have one of Bob’s keys, you know a box came from him because Bob’s keys will open Bob’s padlocks (let’s imagine a world where padlocks cannot be forged even if you know the key). Bob then sends the box to Alice.
In order for Alice to open the box, she needs two keys: her private key that opens her own padlock, and Bob’s well-known key. If Bob’s key doesn’t open the second padlock, then Alice knows that this is not the box she was expecting from Bob, it’s a forgery.

This bidirectional guarantee around identity is known as mutual authentication.

## Examples
### nacl.public.Box
The `Box` class uses the given public and private (secret) keys to derive a shared key, which is used with the nonce given to encrypt the given messages and to decrypt the given ciphertexts. The same shared key will be generated from both pairing of keys, so given two keypairs belonging to Alice (pkalice, skalice) and Bob (pkbob, skbob), the key derived from (pkalice, skbob) will equal that from (pkbob, skalice).

This is how the system works:

In [2]:
import nacl.utils
from nacl.public import PrivateKey, Box

# Generate Bob's private key, which must be kept secret
skbob = PrivateKey.generate()

In [3]:
skbob.encode(encoder = nacl.encoding.HexEncoder)

b'cdeaed19f7fa35500d8bfbec52db142d2e25948463354f7caa2929c102933497'

In [4]:
# Bob's public key can be given to anyone wishing to send Bob an encrypted message
pkbob = skbob.public_key
pkbob.encode(encoder = nacl.encoding.HexEncoder)

b'e497c2a6756ff3949ab6255dd7ea380874e3dd2293d077233276be728cdade37'

In [5]:
# Alice does the same and then Alice and Bob exchange public keys
skalice = PrivateKey.generate()
pkalice = skalice.public_key

In [6]:
# Bob wishes to send Alice an encrypted message so Bob must make a Box with his private key and Alice's public key
bob_box = Box(private_key = skbob, public_key = pkalice)
bob_box.encode(encoder = nacl.encoding.HexEncoder)

b'52d50ca3eb30178f67d2aaafc1f98aaef4ba4532e072d3fbe35c564785ca3cea'

In [7]:
# This is our message to send, it must be a bytestring as Box will treat it as just a binary blob of data.
message = b"I am a box message"
message

b'I am a box message'

PyNaCl can automatically generate a random nonce for us, making the encryption very simple:

In [8]:
# Encrypt our message, it will be exactly 40 bytes longer than the original message as it stores authentication
# information and the nonce alongside it.
encrypted = bob_box.encrypt(message)
print("""encrypted: {}
Length of plaintext (message): {}
Length of cyphertext (encrypted): {}""".format(encrypted, len(message), len(encrypted)))

encrypted: b'a\xe4m\n\xb2\x11\xaa6\x88{\t}^\xb5\x80\x14\xd0\xac\x8eo\xde"\xf3GZ\xdc\xb1@E\xd4\xbf.\xd3\xa4\xb2\xb1\xa6J\xe3\x0b\xc0]07_,\xea4F[\xb6\xcb_<2\x89\x07x'
Length of plaintext (message): 18
Length of cyphertext (encrypted): 58


However, if we need to use an explicit nonce, it can be passed along with the message:

In [9]:
# This is a nonce, it *MUST* only be used once, but it is not considered secret and can be transmitted or stored
# alongside cipher text. A good source of nonces are just sequences of 24 random bytes.
nonce = nacl.utils.random(Box.NONCE_SIZE)
nonce

b'\xa3\xb3\x97\x10H\xf1K\xac\x96\xfd\xa1p/SK\x9cRrb\xe5\x9d=\xa9\x1d'

In [10]:
encrypted = bob_box.encrypt(message, nonce)
encrypted

b'\xa3\xb3\x97\x10H\xf1K\xac\x96\xfd\xa1p/SK\x9cRrb\xe5\x9d=\xa9\x1d\xb8\x96\xdc$XcY\xda\xa6\x14X\x9a]\xe4NZq"\x0fP\xea\xf8\xf9\xea\xedF+\xbf\xa6Ev\x0eGD'

In [11]:
type(encrypted)

nacl.utils.EncryptedMessage

Finally, the message is decrypted (regardless of how the nonce was generated):

In [12]:
# Alice creates a second box with her private key to decrypt the message
alice_box = Box(private_key = skalice, public_key = pkbob)

# Decrypt our message, an exception will be raised if the encryption was tampered with or there was otherwise an error.
plaintext = alice_box.decrypt(encrypted)
print(plaintext.decode("utf-8"))

I am a box message
